In [2]:
# 必要なライブラリのimport
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
import os
import re
import glob
import shutil
import gc
import pyarrow as pa
import dask as da
from pathlib import Path

# matplotlibの日本語化対応
import japanize_matplotlib

# データフレーム表示用関数
from IPython.display import display

# 表示オプション調整
# numpyの浮動小数点の表示精度
np.set_printoptions(suppress=True, precision=4)

# pandasでの浮動小数点の表示精度
pd.options.display.float_format = '{:.4f}'.format

# データフレームですべての項目を表示
pd.set_option("display.max_columns",None)

# グラフのデフォルトフォント指定
plt.rcParams["font.size"] = 14

# グラフの表示方法調整
sns.set(rc={'figure.figsize':(12,5)});
plt.figure(figsize=(12,5));

# 乱数の種
random_seed = 123

<Figure size 864x360 with 0 Axes>

In [3]:
# タイマー
# https://github.com/nyk510/vivid/blob/master/vivid/utils.py
from contextlib import contextmanager
from time import time

class Timer:
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' '):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)

## 機械学習とは
機械学習とは、ざっと言ってしまうとあるデータ X を入力として対応する予測値 y を取り出すような対応関係を作成することです。

例：タイタニック号で、乗客が生きるか死ぬかを予測する問題だと X は乗客の年齢, 性別, 船室のグレード… など乗客に紐づく情報のことを指します。通常、この情報のことを特徴量とよびます。

特徴量 X と 予測値 y が用意できれば学習用データ (X - y の関係がわかっているデータ) を元にして X をいれて y になるようにモデルを調整する。この調整の段階を学習とよびます。学習には様々なアルゴリズムがあるが、X, y を用意しなくてはならない部分は基本的に変わらないです。

# 今回のコンペに関して

コンペ自体のデータサイズが大きいのでデータハンドリングが僕たちに取っては肝になりそうです（この辺をよく知っている人教えてください！）

### データの読み込みに関して
* 形式を変えたデータセットをpandasで読み込む
  * [`feather`形式](https://www.kaggle.com/datasets/munumbutt/amexfeather) or [`Parquet`形式](https://www.kaggle.com/datasets/odins0n/amex-parquet)
* [`pyspark`](https://www.kaggle.com/code/rakkaalhazimi/export-large-dataset-to-spark) or [`dask`](https://docs.dask.org/en/latest/dataframe.html) で読み込む

### サイズが大きい場合の対処方法は以下のkaggle notebookを参考にしてみてください

[> How to Work with BIG Datasets on 16G RAM (+Dask)](https://www.kaggle.com/code/yuliagm/how-to-work-with-big-datasets-on-16g-ram-dask)

上記notebookの内容概要
* TIP 1 - 使用していない変数を [`delete`](https://www.sejuku.net/blog/74447) するか & gc.collect()` で[ガベージコレクション](https://techacademy.jp/magazine/19437)(不要になったメモリ領域を開放して再利用する機能)をする
* TIP 2 - データセット内の各カラムのデータタイプを予め定義しておく
  * eg: 本来 float64 だったものを -> float32 と定義してサイズを圧縮する
* TIP 3 - 読み込むデータセットを選択する (including generating your own subsamples)
  * 読み込む行数を選ぶ
  * (`skiprows`) で[読み飛ばす行数を指定する](https://bit.ly/3O90Ze7)
  * 読み飛ばすリストを作成して、読み飛ばす（以下例）
    * ```
      skiplines = np.random.choice(np.arange(1, lines), size=lines-1-1000000, replace=False)
      skiplines=np.sort(skiplines)
      train = pd.read_csv('../input/train.csv', skiprows=skiplines, dtype=dtypes)
      ```
* TIP 4 - バッチ処理をする
  * ひとまとまりのデータに対して、一連の処理を連続で実行する処理方式のこと。大きなデータに関しても、設定したデータ数(チャンク)ごとに処理をする
* TIP 5 - 特定のカラムのみ `import` する
  * 450,000行 × 150カラム より 100万行 × 2カラム の方がメモリ消費が少ないことは容易に想像できる
* TIP 6 - groupby などの処理をするときも一部カラムの一部データのみにするˆ
* TIP 7 - `Dask` を使用する。
  * [DaskについてのQiita記事](https://qiita.com/simonritchie/items/e174f243bc03fb25462e)

## コンペの概要
* コンペ名：[American Express - Default Prediction](https://www.kaggle.com/competitions/amex-default-prediction)

* 目的：毎月の顧客プロファイルから、顧客がクレジットカードの残高分を将来返済しない確率を予測すること
  * ターゲットのバイナリ変数は、最新のクレジットカード明細書から18ヶ月間のパフォーマンスウィンドウを観察することによって計算され、顧客が最新の明細書の日付から120日以内に支払額を支払わない場合、デフォルトとみなされる。

* 評価方法(Evaluation)
  * このコンペではクレカのデフォルト率を予測する。サブミットはちょっと特殊な評価方法で評価される。以下の通り:
    * ```
      M = 0.5*(G+D)  (*G = 正規化ジニ係数, D = デフォルト率 4% )
      ```
      機械学習における `正規化ジニ係数` は経済学などで使用される ジニ係数とは違うので注意です。以下記事を参照にしてみてください
      * [機械学習のモデル評価、説明可能性のための指標　その１。ジニ係数とAUC](https://qiita.com/Derek/items/4ded249f7a75f8da176c)
      * [DataRobot](https://docs.datarobot.com/ja/docs/modeling/reference/model-detail/opt-metric.html#gini-coefficient)
      * [GINI and AUC relationship](https://stats.stackexchange.com/questions/342329/gini-and-auc-relationship)
      * [Why use Normalized Gini Score instead of AUC as evaluation?](https://stats.stackexchange.com/questions/306287/why-use-normalized-gini-score-instead-of-auc-as-evaluation)

# データ管理環境整備

≒ディレクトリ整備

In [5]:
# # input_dir（input directory） を作ります
current_note_path = os.path.dirname(os.path.abspath('__file__'))
INPUT_DIR = os.path.join(current_note_path, "data")

# INPUT_DIRがまだ作られていなければ作成
if not os.path.isdir(INPUT_DIR):
    os.mkdir(INPUT_DIR)

# output_dir(output directory) を作ります
OUTPUT_DIR = os.path.join(current_note_path, 'outputs')

# OUTPUT_DIRがまだ作られていなければ作成
if not os.path.isdir(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

In [4]:
# 一度やったらコメントアウトします

# # csvファイルを `data` ディレクトリ（=フォルダー） に移動させます
# unique_dir_names = []
# for f in Path(f'{current_note_path}').rglob('*.csv'):
#     unique_dir_names.append(f)

# for file in list(set(unique_dir_names)):
#     print(f'moved file: {file}')
#     shutil.move(f'{file}', f'{INPUT_DIR}')

moved file: /Users/satoshiido/Documents/coding_general/kaggle/amex-default-prediction/test_data.csv
moved file: /Users/satoshiido/Documents/coding_general/kaggle/amex-default-prediction/train_labels.csv
moved file: /Users/satoshiido/Documents/coding_general/kaggle/amex-default-prediction/train_data.csv
moved file: /Users/satoshiido/Documents/coding_general/kaggle/amex-default-prediction/sample_submission.csv


In [6]:
# csv(or any file)を簡単に読み込めるようにする関数
# csv を読み取る関数を設定したあげると、pathや拡張子を書かずに読み込めるので入力が楽になります

# 通常のcsvファイルを読み取るときは以下
def read_csv(name, **kwrgs):
    path = os.path.join(INPUT_DIR, name + '.csv')
    print(f'Load: {path}')
    return pd.read_csv(path, **kwrgs)

# 今回はparquet形式を読み込むので別途用意
def read_parquet(name, **kwrgs):
    path = os.path.join(INPUT_DIR, name + '.parquet')
    print(f'Load: {path}')
    return pd.read_parquet(path, **kwrgs)

# データ読み込み

前記の通り、データサイズが大きく、安易に `pandas` などを使ってもローカルではメモリが足りないので、ここではcsv形式を `parquet` 形式に変えてimport します

(*本来は自分でcsv -> parquet形式に変える必要がありますが、心優しい人がparquet形式にしてくれているのでそれを参照します。)
* [該当データダウンロードページ](https://www.kaggle.com/datasets/odins0n/amex-parquet)
* [Load Parquet Files with Low Memory](https://www.kaggle.com/code/odins0n/load-parquet-files-with-low-memory)
* [parquetデータを使用したEDA](https://www.kaggle.com/code/odins0n/amex-default-prediction-detailed-eda)

In [9]:
# parquet で import をするのでもメモリが足りない人はまた教えてください！

test_df = read_parquet('test_data')
train_df = read_parquet('train_data')
train_labels = read_csv('train_labels')
sample_sub = read_csv('sample_submission')

Load: /Users/satoshiido/Documents/coding_general/kaggle/amex-default-prediction/data/test_data.parquet
Load: /Users/satoshiido/Documents/coding_general/kaggle/amex-default-prediction/data/train_data.parquet
Load: /Users/satoshiido/Documents/coding_general/kaggle/amex-default-prediction/data/train_labels.csv
Load: /Users/satoshiido/Documents/coding_general/kaggle/amex-default-prediction/data/sample_submission.csv


In [10]:
# ガベージコレクション
gc.collect()

0

In [11]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5531451 entries, 0 to 5531450
Columns: 191 entries, customer_ID to target
dtypes: float32(185), int64(2), object(4)
memory usage: 4.1+ GB


In [ ]:
# 以下のカラムは頻出と考えられ、毎回入力するのはめんどくさいので、ポップアップされるように定義します
customer_ID = 'customer_ID'

# 各テーブルの概説

* データについて
  * データセットには各顧客の特徴が各明細書日付ごとに集約されたものが含まれている。特徴は匿名化され、正規化されており、以下のカテゴリに分類されている（カラムの prefix を見ると分かる）:
    * D_*: Delinquency variables
    * S_*: Spend variables
    * P_*: Payment variables
    * B_*: Balance variables
    * R_*: Risk variables
  * 以下のカラムはカテゴリ変数である:
    * B_30, B_38, D_63, D_64, D_66, D_68, D_114, D_116, D_117, D_120, D_126

# EDA (データ確認)

> これ以降のデータハンドリングは Dask packageを使おうと思います（僕もまだ勉強中です）

この辺の記事を読んでなんとなく理解してみてください！
* [PythonのDaskをしっかり調べてみた（大きなデータセットを快適に扱う）](https://qiita.com/simonritchie/items/e174f243bc03fb25462e)

> EDA はこの辺もまずは参考にしてみてもいいと思います
* [AMEX EDA (Comparison of training and test data)](https://www.kaggle.com/code/onodera1/amex-eda-comparison-of-training-and-test-data)

In [17]:
train_df.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.9385,0.0017,0.0087,1.0068,0.0092,0.1240,0.0088,0.0047,NaN,NaN,0.0006,0.0810,0.7089,0.1706,0.0062,0.3586,0.5254,0.2557,NaN,0.0639,0.0594,0.0065,0.1487,1.3359,0.0082,0.0014,0.2073,0.7365,0.0962,NaN,0.0234,0.0028,0.0083,1.0015,0.0083,0.1613,0.1483,0.9230,0.3546,0.1520,0.1181,0.0019,0.1586,0.0657,0.0184,0.0636,0.1996,0.3082,0.0164,0.4016,0.0911,CR,O,0.0071,0.0077,NaN,0.6530,0.0085,NaN,0.0047,6.0000,0.2720,0.0084,0.5152,0.0026,0.0090,0.0048,0.0083,0.1194,0.0048,0.1083,0.0509,NaN,0.0076,0.0804,0.0691,NaN,0.0043,0.0076,NaN,0.0077,0.0003,0.0016,0.0042,0.0014,NaN,0.0023,0.0041,0.0071,0.0025,0.0023,0.0035,0.5066,0.0080,1.0098,0.0847,0.0038,0.0070,0.0004,0.0065,0.0008,0.0051,NaN,0.0000,0.0057,0.0071,NaN,0.0002,0.0089,NaN,1,0.0025,0.0052,0.0066,0.0097,0.0078,0.0024,1.0011,0.0027,0.0075,0.0069,1.5037,1.0061,0.0036,0.0089,0.0039,0.0036,0.0050,0.8941,0.1356,0.9112,0.9745,0.0012,0.7667,1.0087,1.0046,0.8937,NaN,0.6700,0.0100,0.0046,NaN,1.0089,2.0000,NaN,0.0043,NaN,NaN,NaN,1.0073,0.2101,0.6769,0.0079,1.0000,0.2383,0.0000,4.0000,0.2321,0.2363,0.0000,0.7023,0.4343,0.0031,0.6865,0.0087,1.0000,1.0033,1.0078,1.0001,0.0068,NaN,0.0021,0.0060,NaN,0.0043,0.0015,NaN,NaN,NaN,NaN,NaN,0.0024,0.0037,0.0038,NaN,0.0006,0.0006,0.0027,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.9367,0.0058,0.0049,1.0007,0.0062,0.1267,0.0008,0.0027,NaN,NaN,0.0025,0.0694,0.7128,0.1132,0.0062,0.3536,0.5213,0.2233,NaN,0.0653,0.0577,0.0016,0.1497,1.3398,0.0084,0.0020,0.2028,0.7209,0.0998,NaN,0.0306,0.0027,0.0025,1.0090,0.0051,0.1410,0.1435,0.9194,0.3268,0.1562,0.1187,0.0016,0.1485,0.0939,0.0130,0.0655,0.1514,0.2650,0.0177,0.4063,0.0868,CR,O,0.0024,0.0071,NaN,0.6471,0.0022,NaN,0.0039,6.0000,0.1890,0.0040,0.5090,0.0042,0.0078,0.0013,0.0065,0.1406,0.0001,0.1010,0.0405,NaN,0.0048,0.0814,0.0742,NaN,0.0042,0.0053,NaN,0.0019,0.0010,0.0099,0.0076,0.0005,NaN,0.0098,0.0001,0.0060,0.0004,0.0013,0.0078,0.5009,0.0008,1.0095,0.0818,0.0003,0.0078,0.0043,0.0023,0.0095,0.0038,NaN,0.0000,0.0076,0.0067,NaN,0.0011,0.0059,NaN,1,0.0084,0.0090,0.0019,0.0099,0.0060,0.0022,1.0068,0.0025,0.0068,0.0028,1.5036,1.0058,0.0006,0.0004,0.0084,0.0088,0.0032,0.9021,0.1363,0.9199,0.9756,0.0046,0.7860,1.0001,1.0041,0.9068,NaN,0.6686,0.0039,0.0047,NaN,1.0032,2.0000,NaN,0.0087,NaN,NaN,NaN,1.0077,0.1841,0.8223,0.0034,1.0000,0.2472,0.0000,4.0000,0.2435,0.2419,0.0000,0.7070,0.4305,0.0013,0.6864,0.0008,1.0000,1.0084,1.0043,1.0083,0.0044,NaN,0.0010,0.0048,NaN,0.0075,0.0049,NaN,NaN,NaN,NaN,NaN,0.0040,0.0032,0.0050,NaN,0.0096,0.0055,0.0092,0
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.9542,0.0915,0.0217,1.0097,0.0068,0.1240,0.0076,0.0094,NaN,NaN,0.0076,0.0688,0.7209,0.0605,0.0033,0.3347,0.5246,0.1894,NaN,0.0670,0.0566,0.0051,0.1520,1.3372,0.0094,0.0074,0.2066,0.7380,0.1341,NaN,0.0484,0.0101,0.0005,1.0092,0.0070,0.1122,0.1370,1.0020,0.3041,0.1538,0.1145,0.0063,0.1395,0.0848,0.0567,0.0706,0.3059,0.2122,0.0640,0.4068,0.0940,CR,O,0.0019,0.0036,NaN,0.6458,0.0004,Na

In [18]:
test_df.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-02-19,0.6313,0.0019,0.0107,0.8145,0.0075,0.1687,0.0100,0.0023,0.1132,NaN,0.0076,0.1164,0.0045,0.0095,0.0052,NaN,0.4744,0.6265,NaN,0.1746,1.2507,1.0039,NaN,0.0087,0.0010,0.1036,0.1190,NaN,-0.0029,NaN,0.0028,0.0044,1.0069,1.0051,0.0028,0.1056,0.1192,0.3217,0.1146,NaN,NaN,0.0034,0.2059,NaN,0.0102,NaN,0.0275,0.5887,0.0105,1.8819,0.0518,CR,NaN,0.0083,0.0087,NaN,0.5921,0.0091,NaN,0.0050,NaN,0.1905,0.0027,0.4094,0.0100,NaN,0.0052,NaN,0.0147,0.0043,0.4006,1.1348,0.1789,0.9595,0.2177,0.2046,NaN,0.0098,0.0077,0.0029,0.0069,0.0017,0.0044,NaN,0.0020,NaN,0.0025,0.0003,0.0012,0.0042,0.0046,0.0023,0.5082,0.0051,1.0029,0.1338,0.0043,NaN,0.0089,0.0041,0.0096,0.0069,NaN,0.0000,0.0037,0.0099,NaN,0.0089,0.0030,NaN,1,0.0090,0.0034,0.0022,0.0038,0.0029,0.0046,1.0038,0.0067,0.0075,0.0002,NaN,0.0088,0.0008,0.0044,0.0041,0.0084,0.0096,0.9811,0.1349,0.9547,0.9711,0.0066,0.0024,NaN,NaN,NaN,NaN,NaN,0.0081,0.0080,NaN,NaN,1.0000,NaN,0.0051,NaN,NaN,NaN,1.0041,0.2742,0.0043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0062,NaN,NaN,0.0097,NaN,NaN,NaN,NaN,0.0071,0.0060,NaN,NaN,NaN,NaN,NaN,NaN,0.0047,NaN,NaN,NaN,0.0083,NaN
1,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-03-25,0.5870,0.0053,0.0110,0.8108,0.0018,0.2414,0.0002,0.0091,0.1230,NaN,0.0066,0.1349,0.0095,0.0263,0.0090,NaN,0.4736,0.6117,NaN,0.1771,0.2342,1.0093,NaN,0.0031,0.0055,0.1012,0.1210,NaN,0.0312,NaN,0.0065,0.0100,1.0057,1.0004,0.0005,0.1606,0.1215,0.1707,0.1841,NaN,0.0283,0.0072,0.4033,0.0145,0.0251,NaN,0.0548,0.6229,0.0217,2.0433,0.0576,CR,NaN,0.0020,0.0008,NaN,0.5909,0.0002,NaN,0.0016,NaN,0.1902,0.0018,0.5614,0.0081,NaN,0.0084,0.0006,0.0119,0.0023,0.6059,0.2084,0.1795,0.9617,0.2165,0.2066,NaN,0.0045,0.0031,NaN,0.0037,0.0055,0.0015,0.0010,0.0064,NaN,0.0072,0.0031,0.0059,0.0041,0.0096,0.0012,0.5087,0.0073,1.0038,0.1446,0.0053,NaN,0.0015,0.0021,0.0022,0.0055,NaN,0.0000,0.0064,0.0075,NaN,0.0008,0.0075,NaN,1,0.0051,0.0024,0.0066,0.0017,0.0044,0.0006,1.0046,0.0044,0.0012,0.0067,NaN,0.0043,0.0081,0.0079,0.0077,0.0063,0.0070,0.9670,0.1385,0.9570,0.9750,0.0046,0.0065,0.0030,0.0088,NaN,NaN,0.0047,0.0001,0.0155,NaN,1.0077,1.0000,NaN,0.0006,NaN,NaN,NaN,1.0045,0.3536,0.2766,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0038,0.0088,0.0056,0.0074,NaN,0.0076,0.0081,NaN,0.0072,0.0013,NaN,NaN,NaN,NaN,NaN,0.0001,0.0049,0.0090,NaN,0.0037,0.0038,0.0015
2,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-04-25,0.6091,0.0033,0.0164,1.0046,0.0001,0.2670,0.0042,0.0042,0.1253,NaN,0.0096,0.1361,0.0070,0.0600,0.0083,NaN,0.4718,0.6222,NaN,1.4171,0.1270,1.0046,NaN,0.0061,0.0000,0.1090,0.1131,NaN,0.0355,NaN,0.0034,0.0041,0.0064,1.0080,0.0049,0.2101,0.1201,0.1728,0.2540,NaN,0.0337,0.0060,0.3937,0.0229,0.0289,0.4237,0.0837,0.6180,0.0393,1.6419,0.0507,CR,NaN,0.0088,0.0062,NaN,0.5914,0.0061,NaN,0.0023,NaN,-0.0263,0.0072,0.6924,0.0081,NaN,0.0026,0.0017,0.0156,0.0025,0.6019,0.1158,0.1824,0.9638,0.2235,0.2046,NaN,0.0041,0.0085,NaN,0.0024,0.0008,0.0076,0.0087,0.0083,NaN,0.0065,

In [19]:
train_labels.head()

,customer_ID,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0


## 欠損値・異常値確認

In [ ]:
a

# 前処理 + 特徴量エンジニアリング

## カテゴリ変数の変換

## データマージ

# 学習・予測・サブミットファイル作成

## モデル構築

## モデル評価

## サブミットファイル作成